In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# Connect to Comprehend to get Sentiment
import boto3

In [ ]:
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz .
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz .

In [ ]:
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz .
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz .

In [2]:
df1 = pd.read_csv('amazon_reviews_us_Major_Appliances_v1_00.tsv.gz',
                 sep='\t',error_bad_lines=False,warn_bad_lines=False)
print('Rows: {0}, Columns: {1}'.format(df1.shape[0],df1.shape[1]))

Rows: 96834, Columns: 15


In [3]:
df2 = pd.read_csv('amazon_reviews_us_Watches_v1_00.tsv.gz',
                 sep='\t',error_bad_lines=False,warn_bad_lines=False)
print('Rows: {0}, Columns: {1}'.format(df2.shape[0],df2.shape[1]))

Rows: 960204, Columns: 15


In [4]:
df3 = pd.read_csv('amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz',
                 sep='\t',error_bad_lines=False,warn_bad_lines=False)
print('Rows: {0}, Columns: {1}'.format(df3.shape[0],df3.shape[1]))

Rows: 904004, Columns: 15


In [5]:
df4 = pd.read_csv('amazon_reviews_us_Furniture_v1_00.tsv.gz',
                 sep='\t',error_bad_lines=False,warn_bad_lines=False)
print('Rows: {0}, Columns: {1}'.format(df4.shape[0],df4.shape[1]))

Rows: 791673, Columns: 15


In [6]:
columns = ['customer_id','review_id','product_title','product_category','star_rating','verified_purchase','review_headline','review_body']

In [7]:
df1 = df1[columns].dropna()
df2 = df2[columns].dropna()
df3 = df3[columns].dropna()
df4 = df4[columns].dropna()

In [8]:
print('Rows: {0}, Columns: {1}'.format(df1.shape[0],df1.shape[1]))
print('Rows: {0}, Columns: {1}'.format(df2.shape[0],df2.shape[1]))
print('Rows: {0}, Columns: {1}'.format(df3.shape[0],df3.shape[1]))
print('Rows: {0}, Columns: {1}'.format(df4.shape[0],df4.shape[1]))

Rows: 96826, Columns: 8
Rows: 960047, Columns: 8
Rows: 903944, Columns: 8
Rows: 791536, Columns: 8


In [9]:
data = pd.concat([df1,df2,df3,df4])
print('Rows: {0}, Columns: {1}'.format(data.shape[0],data.shape[1]))

Rows: 2752353, Columns: 8


In [10]:
del df1, df2, df3, df4

In [12]:
data1 = data.groupby(['customer_id']).count()[['review_id','star_rating']].sort_values(by='review_id',ascending=False)
data1.shape[0]

1904021

In [21]:
data2 = data1.loc[data1['review_id'] >=30]
customer = data2.index
data3 = data.loc[data['customer_id'].isin(customer)]
data3.shape[0]

27106

In [22]:
data3.head()

,customer_id,review_id,product_title,product_category,star_rating,verified_purchase,review_headline,review_body
2477,34553562,R2HZ5Z39CO33P3,Samsung WF42H5200AW Energy Star 4.2 Cu. Ft. Fr...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Samsung WF42H200AW has earned a spot on Bo...
2502,34553562,R1XP9M6NQK0NHT,"Frigidaire FAFS4474LW Front Load Steam Washer,...",Major Appliances,5,N,"Top 5 Front Loading Washing Machines under $1,...",The Frigidaire FAFS4474LW Front Load Steam Was...
2507,34553562,RL2VDBGE8VOE6,EWFLS70JIW Wave-Touch Series 4.42 cu. ft. Capa...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Electrolux EWFLS70JIW has earned a spot on...
2539,34553562,R205E9DIY5T2LA,GE GFWH1400DWW 4 Cu. Ft. White Stackable Front...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine under $1,5...",The Speed Queen AFN50FS has earned a spot on B...
2550,34553562,R2EX9UU9KCFTLQ,AFN50FS 27&quot; Front-Load Washer with 2.84 c...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Speed Queen AFN50FS has earned a spot on B...


In [24]:
session = boto3.Session(region_name='us-east-1')
client = session.client('comprehend')

In [77]:
step = 25 
index = []
result = []
positive = []
negative = []
neutral = []
mixed = []
for i in range(0,data3.shape[0],step): 
    print('***',i,i+step) 
    review = list((data3.iloc[i:i+step]['review_headline'] + ' - ' + data3.iloc[i:i+step]['review_body'].str.slice(0,4000)).values) 
    sentiment = client.batch_detect_sentiment(TextList=review,LanguageCode='en') 
    for s in sentiment['ResultList']: 
        index.append(s['Index']+i)
        result.append(s['Sentiment'])
        positive.append(round(s['SentimentScore']['Positive'],4))
        negative.append(round(s['SentimentScore']['Negative'],4))
        neutral.append(round(s['SentimentScore']['Neutral'],4))
        mixed.append(round(s['SentimentScore']['Mixed'],4))

*** 0 25
*** 25 50
*** 50 75
*** 75 100
*** 100 125
*** 125 150
*** 150 175
*** 175 200
*** 200 225
*** 225 250
*** 250 275
*** 275 300
*** 300 325
*** 325 350
*** 350 375
*** 375 400
*** 400 425
*** 425 450
*** 450 475
*** 475 500
*** 500 525
*** 525 550
*** 550 575
*** 575 600
*** 600 625
*** 625 650
*** 650 675
*** 675 700
*** 700 725
*** 725 750
*** 750 775
*** 775 800
*** 800 825
*** 825 850
*** 850 875
*** 875 900
*** 900 925
*** 925 950
*** 950 975
*** 975 1000
*** 1000 1025
*** 1025 1050
*** 1050 1075
*** 1075 1100
*** 1100 1125
*** 1125 1150
*** 1150 1175
*** 1175 1200
*** 1200 1225
*** 1225 1250
*** 1250 1275
*** 1275 1300
*** 1300 1325
*** 1325 1350
*** 1350 1375
*** 1375 1400
*** 1400 1425
*** 1425 1450
*** 1450 1475
*** 1475 1500
*** 1500 1525
*** 1525 1550
*** 1550 1575
*** 1575 1600
*** 1600 1625
*** 1625 1650
*** 1650 1675
*** 1675 1700
*** 1700 1725
*** 1725 1750
*** 1750 1775
*** 1775 1800
*** 1800 1825
*** 1825 1850
*** 1850 1875
*** 1875 1900
*** 1900 1925
*** 1925 

*** 14225 14250
*** 14250 14275
*** 14275 14300
*** 14300 14325
*** 14325 14350
*** 14350 14375
*** 14375 14400
*** 14400 14425
*** 14425 14450
*** 14450 14475
*** 14475 14500
*** 14500 14525
*** 14525 14550
*** 14550 14575
*** 14575 14600
*** 14600 14625
*** 14625 14650
*** 14650 14675
*** 14675 14700
*** 14700 14725
*** 14725 14750
*** 14750 14775
*** 14775 14800
*** 14800 14825
*** 14825 14850
*** 14850 14875
*** 14875 14900
*** 14900 14925
*** 14925 14950
*** 14950 14975
*** 14975 15000
*** 15000 15025
*** 15025 15050
*** 15050 15075
*** 15075 15100
*** 15100 15125
*** 15125 15150
*** 15150 15175
*** 15175 15200
*** 15200 15225
*** 15225 15250
*** 15250 15275
*** 15275 15300
*** 15300 15325
*** 15325 15350
*** 15350 15375
*** 15375 15400
*** 15400 15425
*** 15425 15450
*** 15450 15475
*** 15475 15500
*** 15500 15525
*** 15525 15550
*** 15550 15575
*** 15575 15600
*** 15600 15625
*** 15625 15650
*** 15650 15675
*** 15675 15700
*** 15700 15725
*** 15725 15750
*** 15750 15775
*** 1577

*** 27075 27100
*** 27100 27125


In [80]:
data3['sentiment'] = result
data3['prob_positive'] = positive
data3['prob_negative'] = negative
data3['prob_neutral'] = neutral
data3['prob_mixed'] = mixed

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [81]:
data3.head()

,customer_id,review_id,product_title,product_category,star_rating,verified_purchase,review_headline,review_body,sentiment,prob_positive,prob_negative,prob_neutral,prob_mixed
2477,34553562,R2HZ5Z39CO33P3,Samsung WF42H5200AW Energy Star 4.2 Cu. Ft. Fr...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Samsung WF42H200AW has earned a spot on Bo...,NEUTRAL,0.2429,0.0001,0.7570,0.0
2502,34553562,R1XP9M6NQK0NHT,"Frigidaire FAFS4474LW Front Load Steam Washer,...",Major Appliances,5,N,"Top 5 Front Loading Washing Machines under $1,...",The Frigidaire FAFS4474LW Front Load Steam Was...,NEUTRAL,0.0427,0.0001,0.9572,0.0
2507,34553562,RL2VDBGE8VOE6,EWFLS70JIW Wave-Touch Series 4.42 cu. ft. Capa...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Electrolux EWFLS70JIW has earned a spot on...,NEUTRAL,0.0261,0.0001,0.9738,0.0
2539,34553562,R205E9DIY5T2LA,GE GFWH1400DWW 4 Cu. Ft. White Stackable Front...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine under $1,5...",The Speed Queen AFN50FS has earned a spot on B...,NEUTRAL,0.0862,0.0001,0.9137,0.0
2550,34553562,R2EX9UU9KCFTLQ,AFN50FS 27&quot; Front-Load Washer with 2.84 c...,Major Appliances,5,N,"Top 5 Front Loading Washing Machine Under $1,5...",The Speed Queen AFN50FS has earned a spot on B...,NEUTRAL,0.0486,0.0001,0.9512,0.0


In [82]:
data3.to_csv('data.csv')